In [1]:
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import json
import lxml
import pandas as pd
import pyterrier as pt
import string



In [2]:
!rm -rf ./index

In [3]:
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/caleb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/caleb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
input_file = "Answers.json"
query_file = "topics_1.json"
qrel_file = "qrel_1.tsv"

with open(input_file,'r') as file:
    answers = json.load(file)

with open(query_file,'r') as file:
    queries = json.load(file)

In [5]:
def process_input(data):
    for item in data:
        text = item['Text']
        soup = BeautifulSoup(text, "lxml")
        text = soup.get_text()
        item['Text'] = text

In [7]:
process_input(answers)

/tmp/ipykernel_190772/1494876135.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")
/tmp/ipykernel_190772/1494876135.py:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "lxml")


In [8]:
index_path = "./index"

In [9]:
columns = ['Id','Text','Score']

In [10]:
answer_frame = pd.DataFrame(answers, columns=columns)


In [11]:
answer_frame = answer_frame.rename(columns={'Id': 'docno','Text':'text'})

In [12]:
answer_frame

,docno,text,Score
0,2,Maybe these instructions will help you to get ...,41
1,10,You could use freenx instead of vnc. Freenx tr...,12
2,12,"Yes, X forwarding over ssh is a beautiful thin...",22
3,18,You can use the alternative CD (instead of Ubu...,7
4,19,I've always helped identified people's needs a...,8
...,...,...,...
523723,1509129,"If anyone struggling with this, copy the conte...",1
523724,1509131,Edit the file /etc/systemd/sleep.conf with the...,0
523725,1509140,Had a similar issue withAndroid Smartphone: Hu...,1
523726,1509147,"As described in another answer, the solusion i...",0


In [13]:
pd_indexer = pt.terrier.IterDictIndexer(index_path).index(answer_frame.to_dict(orient='records'))

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


05:37:01.305 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Adding an empty document to the index (16123) - further warnings are suppressed
05:37:27.446 [ForkJoinPool-1-worker-1] WARN org.terrier.structures.indexing.Indexer -- Indexed 4 empty documents


In [14]:
index = pt.IndexFactory.of(pd_indexer)

In [15]:
print(index.getCollectionStatistics().toString())

Number of documents: 523728
Number of terms: 621170
Number of postings: 19910779
Number of fields: 1
Number of tokens: 31763654
Field names: [text]
Positions:   false



In [16]:
#temp deprecated
#queries = pd.read_json(query_file)

In [ ]:
#temp deprecated
# queries.drop(columns=['Body'])

In [18]:
"""def process_queries(queries):
    #this converts the raw json querries into [qid: 'num',query:[token,token,token]]
    processed_queries = pd.DataFrame(columns=['qid', 'query'])

    for index, row in queries.iterrows():
        qid, title = row['Id'], row['Title']
        #THIS IS SOOOO IMPORTANT HOLY CRAP WHY DOES THE "?" CHAR CRASH TERRIER?!?!?!?!?
        title = title.translate(str.maketrans('', '', string.punctuation))
        new_row = {'qid': qid, 'query': title}
        new_row_df = pd.DataFrame([new_row])
        processed_queries = pd.concat([processed_queries, new_row_df], ignore_index=True)
        
    return processed_queries
    """ #deprecated this should work but doesnt...

In [28]:
def process_queries(queries):
    lst_q = []

    for item in queries:
        lst_q.append([item['Id'],
        item['Title'].translate(str.maketrans('', '', string.punctuation))])
    processed_queries = pd.DataFrame(lst_q, columns=["qid", "query"])
    return processed_queries

In [29]:
queries = json.load(open(query_file))
processed_queries = process_queries(queries)

In [59]:
bm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "BM25"},verbose=True,num_results=1000)
dfiz_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIZ"},verbose=True,num_results=1000)
dfic_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFIC"},verbose=True,num_results=1000)
hiemstralm_retr = pt.terrier.Retriever(index,controls = {"wmodel": "Hiemstra_LM"},verbose=True,num_results=1000)
dfree_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFRee"},verbose=True,num_results=1000)
dfreeKlim_retr = pt.terrier.Retriever(index,controls = {"wmodel": "DFReeKLIM"},verbose=True,num_results=1000)
tf_retr = pt.terrier.Retriever(index,controls = {"wmodel": "TF_IDF"},verbose=True,num_results=1000)

In [98]:
bm_res = bm_retr.transform(processed_queries)
dfiz_res = dfiz_retr.transform(processed_queries)

TerrierRetr(DFIZ): 100%|██████████| 5418/5418 [01:25<00:00, 63.55q/s] 


In [ ]:
#bm_retr.transform(processed_queries.loc[0,"query"])

In [61]:
#processed_queries.loc[0,"query"]

'How can I set the Software Center to install software for nonroot users'

In [99]:
pt.io.write_results(bm_res, "res_bm25.txt",format= "trec")
pt.io.write_results(dfiz_res,"res_dfiz.txt",format= "trec")

In [40]:
#evaluation
from pyterrier.measures import *
qrels = pt.io.read_qrels(qrel_file)



In [125]:
bm_eval = pt.Evaluate(bm_res,qrels)
dfiz_eval = pt.Evaluate(dfiz_res,qrels) 

In [102]:
dfiz_eval

{'map': 0.06573296091324747, 'ndcg': 0.18349634876631107}

In [91]:
metrics = ["ndcg", "P_5", "P_10", "P", "map", "bpref","recip_rank"]

In [103]:
res = pt.Experiment(
    [bm_retr,dfiz_retr,dfic_retr,hiemstralm_retr,dfree_retr,dfreeKlim_retr,tf_retr],
    processed_queries,
    qrels,
    eval_metrics=metrics,
    names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"],
    save_dir="./",
    filter_by_qrels=True,
    test='t',
    baseline=6
)

/home/caleb/school/information-retrieval/.venv/lib/python3.12/site-packages/pyterrier/pipelines.py:171: UserWarning: save_dir is set, but the file './BM25.res.gz' already exists. If you are aware of are happy to reuse this file to speed up evaluation, set save_mode='reuse'; if you want to overwrite it, set save_mode='overwrite'. To make this condition an error, use save_mode='error'.
  warn(("save_dir is set, but the file '%s' already exists. If you are aware of are happy to reuse this " % save_file)+
TerrierRetr(BM25): 100%|██████████| 5418/5418 [01:24<00:00, 64.10q/s] 
/home/caleb/school/information-retrieval/.venv/lib/python3.12/site-packages/pyterrier/pipelines.py:171: UserWarning: save_dir is set, but the file './DFIZ.res.gz' already exists. If you are aware of are happy to reuse this file to speed up evaluation, set save_mode='reuse'; if you want to overwrite it, set save_mode='overwrite'. To make this condition an error, use save_mode='error'.
  warn(("save_dir is set, but the f

In [104]:
print(res)

          name       map     bpref       P_5      P_10      P@15      P@20  \
0         BM25  0.070726  0.391480  0.072056  0.056109  0.046659  0.040153   
1         DFIZ  0.065733  0.380561  0.068217  0.052953  0.043189  0.036859   
2         DFIC  0.065375  0.382195  0.066962  0.052529  0.043140  0.036859   
3  Hiemstra_Lm  0.064760  0.381157  0.065116  0.050369  0.041688  0.036139   
4        DFRee  0.066965  0.375575  0.070616  0.053027  0.043202  0.037080   
5    DFreeKlim  0.069691  0.388222  0.071945  0.055131  0.045035  0.038787   
6       TF-IDF  0.070619  0.390821  0.072721  0.056202  0.046425  0.039932   

       P@30     P@100     P@200  ...  P@200 p-value  P@500 +  P@500 -  \
0  0.031635  0.014566  0.009025  ...   5.324495e-03    228.0    177.0   
1  0.029254  0.013547  0.008401  ...   2.186265e-43    347.0    918.0   
2  0.029316  0.013525  0.008427  ...   1.072979e-40    360.0    881.0   
3  0.028781  0.013763  0.008596  ...   1.825037e-18    557.0    821.0   
4  0.02930

In [81]:
import matplotlib.pyplot as plt
import numpy as np

In [122]:
names=["BM25","DFIZ","DFIC","Hiemstra_Lm","DFRee","DFreeKlim","TF-IDF"]

res_df = pd.DataFrame(res)

In [ ]:
res_df


In [107]:
res_df.to_csv('results.tsv', sep='\t', index=False)

In [129]:
per_query = pt.Experiment(
    [bm_retr,tf_retr,dfree_retr],
    processed_queries,
    qrels,
    eval_metrics=["ndcg", "P_5", "bpref","recip_rank"],
    filter_by_qrels=True,
    perquery=True,
    names=["BM25","TF-IDF","DFreeKlim"]
    )

TerrierRetr(BM25):   0%|          | 0/5418 [00:00<?, ?q/s]

TerrierRetr(DFRee): 100%|██████████| 5418/5418 [01:43<00:00, 52.19q/s] 


In [130]:
per_query.to_csv('per_q.tsv',sep='\t', index=False)

In [115]:
#Ad-hoc querry
ad_hoc = "best way to monitor processes"
top_k = 5
top_k_bm = bm_retr.search(ad_hoc).head(top_k)
top_k_df = dfreeKlim_retr.search(ad_hoc).head(top_k)


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00, 38.74q/s]
